### 


# Lab 1: Implementing the α<sub>CE</sub>–λ Prescription

In this lab, we will implement the **α<sub>CE</sub>–λ formalism** by calculating both the binding energy and the λ parameter from the final stellar profiles in our grids.  

---

### Motivation
---
The **post-common-envelope (post-CE) outcome** of a binary system is very sensitive to how we define the boundary between the core and the envelope of the donor star.  

In traditional binary population synthesis codes, the core boundary is often fixed using preset hydrogen mass fractions.

Here, we aim to calculate **binding energy** and **λ values** for **our own definition of the core boundary**. This allows us to explore how different choices affect the envelope ejection and the resulting post-CE orbital separation.  

---
Following the $\alpha_{CE} - \lambda$ prescription we relate the binding energy used for the ejection of the envelope with the change in orbital energy through the efficiency coeffient $a_{CE}$.

$$
    E_{bind} = a_{CE} \, \Delta E_{orb}
$$


The binding energy of the envelope is given by:  

$$
E_{\rm bind} = \int_{M(r^*)}^{M(R)} \Big( - \frac{G m}{r(m)} + \alpha_{th} U(m) \Big) dm
$$

where:
- $ r^* $ = radial coordinate of the core–envelope boundary
- $ R $ = stellar radius, with M(R) = M_{\rm donor}
- $ M(r^*) = M_{\rm core} $, the remaining mass after the envelope is removed  
- $ U(m) $ = internal energy per unit mass  
- $ \alpha_{th} $ = fraction of internal energy contributing to envelope ejection  


The change in orbital energy during CE evolution is:  

$$
\Delta E_{\rm orb} = E_{\rm orb}^{f} - E_{\rm orb}^{i} 
= - \frac{G M_{\rm core} M_2}{2 a_f} + \frac{G M_{\rm donor} M_2}{2 a_0}
$$

where:
- $ M_{\rm donor} $ = donor mass  
- $ M_{\rm core} $ = post-CE core mass  
- $ M_1 $ = companion mass  
- $ a_0, a_f $ = orbital separation before and after CE  

---

In the α<sub>CE</sub>–λ scheme, the binding energy is often expressed as:  

$$
E_{\rm bind} = - \frac{G M_{\rm donor} \, M_{\rm env}}{\lambda \, R_{\rm donor}}
$$


Where the structure parameter $\lambda$ depends on the stellar density of the star. The value of this parameter depends on the evolutionary state and structure of the donor. Generally, in BPS codes the $\lambda$ coeffiecent is estimated using tables of precomputed values for a given mass from the literature ( Dewi & Tauris, 2000, 2001; Xu & Li, 2010; Loveridge et al., 2011 ). These predifined values are very sensitive to the definition of the core mass coordinate but also depend on the prior evolution that the star had. 

---
In **POSYDON**, we have access to the final stellar profiles right before the binary enters CE.  
This allows us to:
1. Calculate λ directly and on-the-fly for a given donor.  
2. Flexibly choose a core–envelope boundary for $X_H = 0.01,\; 0.1,\; \text{or}\; 0.3$.  

By doing this, we can see how the post-CE outcome changes depending on our **core definition**, giving us more physical insight into envelope ejection and binary evolution.  

---

### Goal of this Lab
---
In this first lab, we will:  
1. Build a function to calculate the **binding energy** of the envelope.  
2. Extend it to compute the **λ parameter**.  
(3. Use these results to obtain the **post-CE orbital separation**.)

This functions will later be used in **Lab 2**



---
###  Prerequisites for this Lab

---
<div class="alert alert-info" style="margin-top: 20px">

You are going to run the code for this lab in the `CE_lab.ipynb`, following the instrunctions you are going to fill up the cells of your notebooks. This notebook is where you are going to test your functions, load up the posydon grids and run a sample population. We will use this notebook for both **Lab1** and **Lab2** 

</div>

1. Make sure that you have copied the CE_lab.ipynb in your directory. 
2. Make sure the path to `PATH_TO_POSYDON`, and `PATH_TO_POSYDON_DATA` corresponds with yours. Both of these should be under the directory `POSYDON-shared` 
3. Choose your posydon enviroment for your kernel, and run the first cell.


### Loading the Grids and accessing the profiles 
---
We begin by learning how to load and use Psygrid objects. The Psygrid object is used to incorporate MESA grids into POSYDON. We can use the PsyGrid to access the POSYDON grids and pull up information about the history and the final profiles of every star and binary from our grids. 

Let's begin by loading the HMS-HMS grid. Check in the documentation:  https://posydon.org/POSYDON/v1.0.5/index.html of how to import and load a PSyGrid object. It will take a few minutes to load the grid. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
from posydon.grids.psygrid import PSyGrid
grid = PSyGrid(f"{os.environ['PATH_TO_POSYDON_DATA']}POSYDON_data/HMS-HMS/1e+00_Zsun.h5")

````

Let's explore the PsyGrid object. Let's start by choosing a binary from the grid and navigating it's final profile. Choose an **<span style="color:blue">index < len(grid)</span>** and print out the final profile of the primary. 

In [ ]:
index = 5000
binary = grid[index]
print(binary.final_profile1)

The output is a long array. We will now utilize pandas Dataframe to make the output more readable. 

In [ ]:
primary = pd.DataFrame(binary.final_profile1)
print(primary)

In MESA the profiles correspond to zones in the star, with the zero index being the one of the outermost layer of the star. Make a plot of the internal energy of the star vs. its radius by accesing the keys of primary. Make an additional plot of the H mass fraction vs. radius. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(10, 4))
ax1.plot(np.log10(primary.radius),primary.energy)
ax1.set_ylabel('Energy $[erg]$')
ax1.set_xlabel('log($R/R_\odot$)')
ax2.plot(np.log10(primary.radius),primary.x_mass_fraction_H)
ax2.set_xlabel('log($R/R_\odot$)')
ax2.set_ylabel('$H_1$ fraction')

```
    
</details>
</div>

### Calculating the binding energy
---

Our first step is to built our function that will calculate and return the binding energy. The binding energy is given by the equation above and it consist of two componets: the gravitational energy and the internal energy. The internal energy is given by the final profile and represents the internal energy within each zone.

$$
E_{grav} =   \int_{M(r^*)}^{M(R)} - \frac{G m}{r} dm = \sum_{i} - \frac{G m_i}{r_i} dm_i 
$$

$$
U_{in} = \int_{M(r^*)}^{M(R)}  a_{th} U(m) dm =  \sum_{i} a_{th} U_i dm_i
$$

$$
    E_{bind} = E_{grav} + U_{in} 
$$


This function should take as an input the the hydrogen mass fraction that seperates the core from the envelope. Name this function ` calculate_binding_energy`. After calculating the binding energy we will create a function that calculates the lambda parameter named `calculate_E_lamda_CE` 

This function will take as inputs the variable: `core_definition_H_fraction` and `star` and the thermal parameter `common_envelope_alpha_thermal ` . Let's set `common_envelope_alpha_thermal` = 1 for now. 

The variable star for now will be a pandas array from our grids.

In [ ]:
def calculate_binding_energy(core_definition_H_fraction,star,common_envelope_alpha_thermalm = 1):
    pass 

#### Step 1

Now having the index i we can find all the quantities need for calculating the binding energy and the coeffiecient lambda. 
<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal)</summary></b>

You will need to use constants for this calculation. You can find them in the `posydon/utils/constants.py`. Make sure you import them in the beggining of your notebook 

</details>
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
from posydon.utils import constants as const
```
</details>
</div>

Look at `constants.py`

### Step 2 

We need to include all the inputs we need from the profile such as  `radius`, `mass`,`x_mass_fraction_H`  and `internal_energy`. The profiles include all the cumulative mass, not the shell mass. We need to calculate the mass of each shell: 
$$ dm_i = m_i - m_{i-1} $$
We can compute this using `np.diff`

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal)</summary></b>

The `np.diff(mass)` gives differences $m_{i+1} - m_i $ not $m_{i} - m_{i-1} $. In the mesa profiles when the zone index is increasing we are deeper in the star so $m_{i+1}< m_{i} $. 

</details>
</div>


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
dm = np.concatenate((-1 * np.diff(mass), [mass[-1]]))
```
    
</details>
</div>

### Step 3
We need to find the index for which the Hydrogen mass fraction corresponds is just above the value of the `core_definition` in the profile. This zone will seperate the envelope from the core.

In [1]:
zones = star[star.x_mass_fraction_H > core_definition_H_fraction].index
zone_i = zones[-1]

### Step 4

Looping over all shells up from the boundary `zone_i` to the outermost layer of the star's envelope, calculate the gravitational and internal energy. Finally compute and return the binding energy. 

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal)</summary></b>

Initialize the variable before the loop.  

</details>
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python

    for i in range(zone_i):
        Grav_energy_i = (-const.standard_cgrav * mass[i]
                               * const.Msun * dm[i]*const.Msun
                               / (radius[i]*const.Rsun))
        
        Grav_energy = Grav_energy + Grav_energy_i
        U_i = U_i + internal_energy[i]*dm[i]*const.Msun

    Ebind_i = Grav_energy + common_envelope_alpha_thermal * U_i
    return Ebind_i
                 
```
    
</details>
</div>


### Calculating the Common-Envelope λ Parameter

---

Now that we can compute the **binding energy** of the envelope,  
we want to calculate the **common-envelope (CE) λ parameter** for any arbitrary hydrogen fraction boundary definition.  

The λ parameter is defined as:

$$
\lambda_{\rm CE} = - \frac{G M_{\rm donor} M_{\rm env}}{R \, E_{\rm bind}}
$$


In [11]:
def calculate_E_lamda_CE(core_definition_H_fraction,star):
    pass 

### Step 1

Define all the parameters we need from the star's profile. Calculate the mass of the envelope and core by finding the index of the zone that separates the core from the envelope (in the same way we did it before). 


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python

    zones = star[star.x_mass_fraction_H > core_definition_H_fraction].index
    zone_i = zones[-1]
    
    mass = star.mass
    radius = star.radius
    M_donor = mass[0]
    M_core = mass[zone_i]
    M_envelope = M_donor - M_core
    R = radius[0]

```
</details>
</div>

### Step 2 

Call our previous function `calculate_binding_energy` to compute the binding energy of the envelope. Put everything together to calculate and return λ. 


<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal)</summary></b>

```python
    lambda_CE = - const.standard_cgrav *(M_donor*const.Msun)*(M_envelope*const.Msun)/(R*const.Rsun*E_bind)
    return lambda_CE

```

</details>
</div>

### Calculating the binding energy and lambda for binaries in our grids 

---

We will apply our functions to binaries that undergo unstable mass transfer. 

Every group should select from this spreadsheet ( https://docs.google.com/spreadsheets/d/1t_Kq1XVB75lLrdn1G83n72V_wpnhmrzxQ0Sslww2ZZY/edit?usp=sharing ) a binary and CE efficiency. 

Put your name down next to a set of parameters and write down the outputs from your functions. Compare your results with the rest of the group. 


